# 🛢️ Генератор синтетических скважин

Полнофункциональный notebook для генерации синтетических каротажных данных с использованием:
- Марковских цепей
- Циклов трансгрессии/регрессии
- Эффектов инверсии

## 📋 Содержание:
1. Импорт библиотек
2. Параметры модели
3. Функции генерации
4. Генерация скважины
5. Визуализация
6. Экспорт данных

## 1. Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Настройка стиля
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Библиотеки загружены успешно!")

## 2. Параметры модели

In [ ]:
# Основные параметры
n_depth = 1000  # Количество точек
depth_start = 1000  # Начальная глубина (м)
depth_end = 2000  # Конечная глубина (м)
cycle_length = 200  # Длина цикла
invasion_window = 7  # Окно инверсии
invasion_strength = 0.6  # Сила инверсии
random_seed = 42  # Seed для воспроизводимости

if random_seed is not None:
    np.random.seed(random_seed)

print(f"📊 Параметры: {depth_start}-{depth_end}м, {n_depth} точек")

In [ ]:
# Литологические типы
lithology_states = ['sand', 'shale', 'carbonate_sand', 'coal', 'siltstone']

# Частоты встречаемости
lithology_freq_range = {
    'sand': (0.01, 0.15),
    'shale': (0.35, 0.9),
    'carbonate_sand': (0.05, 0.12),
    'coal': (0.01, 0.08),
    'siltstone': (0.05, 0.15)
}

# Длины серий
series_length_range = {
    'sand': (2, 10),
    'shale': (1, 155),
    'carbonate_sand': (2, 6),
    'coal': (1, 4),
    'siltstone': (2, 20)
}

# Матрица переходов
transition_matrix = np.array([
    [0.6, 0.2, 0.1, 0.05, 0.05],
    [0.3, 0.5, 0.1, 0.05, 0.05],
    [0.2, 0.1, 0.6, 0.05, 0.05],
    [0.1, 0.1, 0.1, 0.6, 0.1],
    [0.2, 0.2, 0.2, 0.05, 0.35]
])

# Свойства литологий
lithology_properties = {
    'sand': {'porosity': (0.25, 0.03), 'sw': (0.3, 0.05), 'gamma_base': 40, 'matrix_density': 2.65, 'color': 'yellow'},
    'shale': {'porosity': (0.15, 0.02), 'sw': (0.8, 0.05), 'gamma_base': 70, 'matrix_density': 2.55, 'color': 'brown'},
    'carbonate_sand': {'porosity': (0.20, 0.02), 'sw': (0.4, 0.05), 'gamma_base': 35, 'matrix_density': 2.71, 'color': 'orange'},
    'coal': {'porosity': (0.35, 0.04), 'sw': (0.2, 0.05), 'gamma_base': 30, 'matrix_density': 1.4, 'color': 'black'},
    'siltstone': {'porosity': (0.18, 0.02), 'sw': (0.5, 0.05), 'gamma_base': 60, 'matrix_density': 2.62, 'color': 'gray'}
}

rho_fluid = 1.0

print("✅ Параметры литологий загружены!")

## 3. Функции генерации

In [ ]:
def sample_initial_prob(freq_range):
    """Генерация начальных вероятностей"""
    freqs = [np.random.uniform(low, high) for (low, high) in freq_range.values()]
    freqs = np.array(freqs)
    freqs /= freqs.sum()
    return freqs


def generate_lithology_markov_cycles(n, states, trans_matrix, initial_prob, series_range, cycle_len=200):
    """Генерация литологии с циклами"""
    lithology = []
    current_lith = np.random.choice(states, p=initial_prob)
    lithology.append(current_lith)

    for i in range(1, n):
        cycle_phase = (i // cycle_len) % 2

        if cycle_phase == 0:  # Трансгрессия
            bias_matrix = np.array([
                [0.4, 0.2, 0.05, 0.05, 0.3],
                [0.2, 0.4, 0.1, 0.05, 0.25],
                [0.1, 0.2, 0.5, 0.05, 0.15],
                [0.1, 0.1, 0.05, 0.7, 0.05],
                [0.15, 0.25, 0.2, 0.05, 0.35]
            ])
        else:  # Регрессия
            bias_matrix = np.array([
                [0.6, 0.1, 0.1, 0.05, 0.15],
                [0.3, 0.3, 0.05, 0.05, 0.3],
                [0.3, 0.1, 0.4, 0.05, 0.15],
                [0.1, 0.1, 0.1, 0.6, 0.1],
                [0.3, 0.2, 0.1, 0.05, 0.35]
            ])

        prev_idx = states.index(current_lith)
        probs = 0.5 * trans_matrix[prev_idx] + 0.5 * bias_matrix[prev_idx]
        probs /= probs.sum()
        current_lith = np.random.choice(states, p=probs)
        lithology.append(current_lith)

    # Растягиваем серии
    lith_expanded = []
    for lith in lithology:
        min_len, max_len = series_range[lith]
        run_len = np.random.randint(min_len, max_len + 1)
        lith_expanded.extend([lith] * run_len)

    return lith_expanded[:n]


def apply_invasion_effect(curve, window=5, alpha=0.5):
    """Эффект инверсии"""
    curve = np.array(curve)
    kernel = np.ones(window) / window
    smooth = np.convolve(curve, kernel, mode="same")
    return alpha * smooth + (1 - alpha) * curve


print("✅ Функции определены!")

## 4. Генерация скважины

In [ ]:
print("🔄 Генерация синтетической скважины...\n")

# Генерация
initial_prob = sample_initial_prob(lithology_freq_range)
lithology = generate_lithology_markov_cycles(
    n_depth, lithology_states, transition_matrix, initial_prob, 
    series_length_range, cycle_len=cycle_length
)

depths = np.linspace(depth_start, depth_end, n_depth)

# Пористость и насыщение
phi, sw = [], []
for lith in lithology:
    props = lithology_properties[lith]
    phi.append(np.random.normal(*props['porosity']))
    sw.append(np.random.normal(*props['sw']))

# Каротаж
gamma_log, rho_log = [], []
for i, lith in enumerate(lithology):
    props = lithology_properties[lith]
    gamma_log.append(np.random.normal(props['gamma_base'], 3))
    rho = phi[i] * rho_fluid + (1 - phi[i]) * props['matrix_density']
    rho += np.random.normal(0, 0.02)
    rho_log.append(rho)

# Инверсия
gamma_inv = apply_invasion_effect(gamma_log, window=invasion_window, alpha=invasion_strength)
rho_inv = apply_invasion_effect(rho_log, window=invasion_window, alpha=invasion_strength)

# DataFrame
df = pd.DataFrame({
    'Depth': depths,
    'Lithology': lithology,
    'Porosity': phi,
    'Sw': sw,
    'Gamma_clean': gamma_log,
    'Gamma_inv': gamma_inv,
    'Rho_clean': rho_log,
    'Rho_inv': rho_inv
})

print("✅ Скважина сгенерирована!")
print(f"Размер: {df.shape[0]} × {df.shape[1]}\n")
df.head()

## 5. Статистика

In [ ]:
# Распределение литологий
print("📊 Распределение литологий:\n")
lith_counts = df['Lithology'].value_counts()
for lith, count in lith_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{lith:20s}: {count:4d} ({percentage:5.2f}%)")

print("\n📈 Средние значения:\n")
print(df.groupby('Lithology')[['Porosity', 'Sw', 'Gamma_inv', 'Rho_inv']].mean().round(3))

## 6. Визуализация каротажа

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(16, 10), sharey=True)

# ГК
axs[0].plot(df['Gamma_clean'], df['Depth'], 'g-', alpha=0.5, label='Чистый')
axs[0].plot(df['Gamma_inv'], df['Depth'], 'darkgreen', linewidth=1.5, label='С инверсией')
axs[0].set_title('Гамма-каротаж', fontweight='bold')
axs[0].set_xlabel('ГК (API)')
axs[0].set_ylabel('Глубина (м)')
axs[0].invert_yaxis()
axs[0].legend()
axs[0].grid(alpha=0.3)

# Плотность
axs[1].plot(df['Rho_clean'], df['Depth'], 'b-', alpha=0.5, label='Чистая')
axs[1].plot(df['Rho_inv'], df['Depth'], 'r-', linewidth=1.5, label='С инверсией')
axs[1].set_title('ГГКП', fontweight='bold')
axs[1].set_xlabel('Плотность (г/см³)')
axs[1].legend()
axs[1].grid(alpha=0.3)

# Пористость
axs[2].plot(df['Porosity'], df['Depth'], 'orange', linewidth=1.5)
axs[2].set_title('Пористость', fontweight='bold')
axs[2].set_xlabel('Пористость')
axs[2].grid(alpha=0.3)

# Насыщение
axs[3].plot(df['Sw'], df['Depth'], 'cyan', linewidth=1.5)
axs[3].set_title('Водонасыщенность', fontweight='bold')
axs[3].set_xlabel('Sw')
axs[3].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Кроссплоты

In [ ]:
palette = {lith: props['color'] for lith, props in lithology_properties.items()}

fig, axs = plt.subplots(1, 3, figsize=(16, 5))

# Gamma vs Density
sns.scatterplot(data=df, x='Gamma_inv', y='Rho_inv', hue='Lithology', 
                palette=palette, alpha=0.7, ax=axs[0])
axs[0].set_title('Gamma vs Density')
axs[0].grid(alpha=0.3)

# Gamma vs Density (по пористости)
sc = axs[1].scatter(df['Gamma_inv'], df['Rho_inv'], c=df['Porosity'], 
                    cmap='viridis', alpha=0.7)
axs[1].set_title('Gamma vs Density (пористость)')
axs[1].set_xlabel('ГК')
axs[1].set_ylabel('Плотность')
axs[1].grid(alpha=0.3)
plt.colorbar(sc, ax=axs[1], label='Пористость')

# Density vs Porosity
sns.scatterplot(data=df, x='Rho_inv', y='Porosity', hue='Lithology', 
                palette=palette, alpha=0.7, ax=axs[2])
axs[2].set_title('Density vs Porosity')
axs[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Экспорт данных

In [ ]:
# CSV
csv_file = f'synthetic_well_{depth_start}_{depth_end}m.csv'
df.to_csv(csv_file, index=False)
print(f"✅ CSV: {csv_file}")

# Excel
excel_file = f'synthetic_well_{depth_start}_{depth_end}m.xlsx'
with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Data', index=False)
    df.groupby('Lithology').agg({
        'Porosity': ['mean', 'std'],
        'Sw': ['mean', 'std'],
        'Gamma_inv': ['mean', 'std'],
        'Rho_inv': ['mean', 'std']
    }).round(3).to_excel(writer, sheet_name='Stats')

print(f"✅ Excel: {excel_file}")
print("\n📁 Экспорт завершен!")